### Simulaciones

Estos son los parámetros que le pasaremos a nuestos agentes para crear las simulaciones, los siguientes parametros tienen valores aleatorios que usamos en la mayoria de las simulaciones, sin embargo, luego los fuimos motificando para observar ciertos comportamientos.

In [1]:
# Definimos los parametros de los agentes trabajadores
MIN_PS_AGENTS = 10          # Numero minimo de resolucion de problemas de los agentes
MAX_PS_AGENTS = 100         # Numero maximo de resolucion de problemas de los agentes
MIN_FRIENDSHIP = 0.0        # Minimo de amistad entre los agentes
MAX_FRIENDSHIP = 1.0        # Maximo de amistad entre los agentes
MIN_MIN_ENERGY = 10         # Minima energia de los agentes para trabajar
MAX_MIN_ENERGY = 100        # Maxima energia de los agentes para trabajar
MIN_MOTIVATION = 10         # Motivacion minima para que los trabajadores se sientan motivados
MIN_LAZZY = 0.0             # Pereza minima de los agentes
MAX_LAZZY = 0.4             # Pereza maxima de los agentes 

# Definimos los parametros del Project Manager
RISKY = 0.2                 # Cuan arriesgado es el project manager
MIN_WORK_PROB = 0           # Probabilidad minima de que elija trabajar él antes de mandar a un trabajador
MAX_WORK_PROB = 0.2  
MIN_COOPERATION = 0.5       # Probabilidad minima de que elija mandar a cooperar en tareas dificiles, 1 - P seria la probabilidad de que reasigne en vez de cooperar
MAX_COOPERATION = 1.0
MIN_EXPLORATION = 0.0       # Probabilidad minima de que tome decisiones poco usuales
MAX_EXPLORATION = 0.2 

# Definimos algunos parametros para las tareas
MIN_DURATION = 5            # Duracion minima de una tarea
MAX_DURATION = 100         
INTERVAL = 1.2              # Porcentaje de tiempo entre el start y el deadline de la tarea
MIN_RESOURCES = 1           # Cantidad minima de recursos que tiene una tarea
MAX_RESOURCES = 10
MIN_DEPENDENCIES = 0        # Cantidad minima de dependencias que tiene una tarea
MAX_DEPENDENCIES = 10
MIN_REWARD = 1              # Recompensa minima que debe tener una tarea
MAX_REWARD = 100 
MIN_DIFFICULTY = 1          # Dificultad minima que debe tener una tarea
MAX_DIFFICULTY = 100
MIN_PROBLEMS_PROB = 0       # Probabilidad minima de una tarea de generar un problema
MAX_PROBLEMS_PROB = 0.5     

NUMBER_OF_TASKS = 50
NUMBER_OF_WORKERS = 10


### Variaremos los parámetros para ver que comportamiento tienen los agentes en diferentes situaciones

#### - Equipo muy amigable.
Este conjunto de parámetros aumenta la amistad entre los agentes (MIN_FRIENDSHIP y MAX_FRIENDSHIP altos), lo que puede resultar en una mayor cooperación y motivación en el equipo, mejorando el rendimiento general.

In [2]:
param_sets = []
# Parámetros de los agentes trabajadores
friedly_team = {
    MIN_PS_AGENTS : 10 ,         
    MAX_PS_AGENTS : 100,        
    MIN_FRIENDSHIP : 0.8,        # Alta amistad mínima
    MAX_FRIENDSHIP : 1.0,        # Alta amistad máxima
    MIN_MIN_ENERGY : 10 ,        
    MAX_MIN_ENERGY : 100,        
    MIN_MOTIVATION : 50 ,        # Motivación alta para trabajar
    MIN_LAZZY : 0,             
    MAX_LAZZY : 0.2,             # Baja pereza

    # Parámetros del Project Manager       
    RISKY : 0.1,                 # Bajo nivel de riesgo (prefiere estrategias seguras)
    MIN_WORK_PROB : 0  ,         
    MAX_WORK_PROB : 0.1,         
    MIN_COOPERATION : 0.7,       # Alta probabilidad de cooperación en tareas difíciles
    MAX_COOPERATION : 1.0,       
    MIN_EXPLORATION : 0.1,       
    MAX_EXPLORATION : 0.2,       

    # Parámetros de las tareas
    MIN_DURATION : 5  ,          
    MAX_DURATION : 100,          
    INTERVAL : 1.2,              
    MIN_RESOURCES : 1  ,         
    MAX_RESOURCES : 10 ,         
    MIN_DEPENDENCIES : 0  ,      
    MAX_DEPENDENCIES : 10 ,      
    MIN_REWARD : 1  ,            
    MAX_REWARD : 100,            
    MIN_DIFFICULTY : 1  ,        
    MAX_DIFFICULTY : 50 ,        # Dificultad moderada
    MIN_PROBLEMS_PROB : 0.1,    
    MAX_PROBLEMS_PROB : 0.5,     

    NUMBER_OF_TASKS : 50,
    NUMBER_OF_WORKERS : 10
}
param_sets.append(friedly_team)

#### - Equipo con baja amistad 
Este conjunto simula un equipo con baja amistad (MIN_FRIENDSHIP y MAX_FRIENDSHIP bajos), lo que puede dificultar la cooperación y llevar a problemas en la ejecución de tareas.

In [3]:
# Parámetros de los agentes trabajadores
not_friendly_team = {
    MIN_PS_AGENTS : 10 ,         
    MAX_PS_AGENTS : 100,         
    MIN_FRIENDSHIP : 0.0,        # Baja amistad mínima
    MAX_FRIENDSHIP : 0.2,        # Baja amistad máxima
    MIN_MIN_ENERGY : 10 ,        
    MAX_MIN_ENERGY : 100,        
    MIN_MOTIVATION : 20 ,        # Motivación mínima
    MIN_LAZZY : 0.2,             
    MAX_LAZZY : 0.4,             # Alta pereza

    # Parámetros del Project Manager       
    RISKY : 0.3,                 # Moderadamente arriesgado
    MIN_WORK_PROB : 0.1,         
    MAX_WORK_PROB : 0.3,         
    MIN_COOPERATION : 0.5,       # Baja cooperación, más reasignación
    MAX_COOPERATION : 0.7,       
    MIN_EXPLORATION : 0.1,       
    MAX_EXPLORATION : 0.3,       

    # Parámetros de las tareas
    MIN_DURATION : 10 ,          
    MAX_DURATION : 120,          
    INTERVAL : 1.1,              
    MIN_RESOURCES : 2  ,         
    MAX_RESOURCES : 15 ,         
    MIN_DEPENDENCIES : 0  ,      
    MAX_DEPENDENCIES : 5  ,      
    MIN_REWARD : 10 ,            
    MAX_REWARD : 80 ,            
    MIN_DIFFICULTY : 30 ,        
    MAX_DIFFICULTY : 90 ,        
    MIN_PROBLEMS_PROB : 0.2,     # Probabilidad moderada de problemas
    MAX_PROBLEMS_PROB : 0.5,     

    NUMBER_OF_TASKS : 50,
    NUMBER_OF_WORKERS : 10
}
param_sets.append(not_friendly_team)

#### - Project Manager muy arriesgado
Este conjunto aumenta el riesgo asumido por el Project Manager (RISKY alto), lo que significa que tomará más decisiones arriesgadas. Esto podría llevar a más oportunidades pero también a más problemas.

In [4]:
# Parámetros de los agentes trabajadores
risky_pm = {
    MIN_PS_AGENTS : 10 ,         
    MAX_PS_AGENTS : 100,         
    MIN_FRIENDSHIP : 0.3,        
    MAX_FRIENDSHIP : 0.7,        
    MIN_MIN_ENERGY : 20 ,        
    MAX_MIN_ENERGY : 90 ,        
    MIN_MOTIVATION : 30 ,        
    MIN_LAZZY : 0.0,             
    MAX_LAZZY : 0.2,             

    # Parámetros del Project Manager      
    RISKY : 0.8,                 # Muy arriesgado
    MIN_WORK_PROB : 0.0,         
    MAX_WORK_PROB : 0.1,         
    MIN_COOPERATION : 0.6,       
    MAX_COOPERATION : 0.9,       
    MIN_EXPLORATION : 0.1,       
    MAX_EXPLORATION : 0.3,       

    # Parámetros de las tareas
    MIN_DURATION : 5  ,          
    MAX_DURATION : 150,          
    INTERVAL : 1.3,              
    MIN_RESOURCES : 1  ,         
    MAX_RESOURCES : 8  ,         
    MIN_DEPENDENCIES : 1  ,      
    MAX_DEPENDENCIES : 10 ,      
    MIN_REWARD : 20 ,            
    MAX_REWARD : 120,            
    MIN_DIFFICULTY : 10 ,        
    MAX_DIFFICULTY : 100,        
    MIN_PROBLEMS_PROB : 0.3,     
    MAX_PROBLEMS_PROB : 0.6,     

    NUMBER_OF_TASKS : 60,
    NUMBER_OF_WORKERS : 12
}
param_sets.append(risky_pm)

#### - Exploración alta en el Project Manager
Aquí el Project Manager tiene una alta probabilidad de tomar decisiones inusuales (EXPLORATION alto). Esto puede llevar a resultados impredecibles y a la búsqueda de estrategias no convencionales.

In [5]:
# Parámetros de los agentes trabajadores
explorator_pm = {
    MIN_PS_AGENTS : 20 ,         
    MAX_PS_AGENTS : 90 ,         
    MIN_FRIENDSHIP : 0.5,        
    MAX_FRIENDSHIP : 0.9,        
    MIN_MIN_ENERGY : 30 ,        
    MAX_MIN_ENERGY : 80 ,        
    MIN_MOTIVATION : 40 ,        
    MIN_LAZZY : 0.1,             
    MAX_LAZZY : 0.3,             

    # Parámetros del Project Manager     
    RISKY : 0.3,                 
    MIN_WORK_PROB : 0.0,         
    MAX_WORK_PROB : 0.1,         
    MIN_COOPERATION : 0.5,       
    MAX_COOPERATION : 0.8,       
    MIN_EXPLORATION : 0.4,       # Muy alta exploración
    MAX_EXPLORATION : 0.6,       

    # Parámetros de las tareas
    MIN_DURATION : 10 ,          
    MAX_DURATION : 90 ,          
    INTERVAL : 1.2,              
    MIN_RESOURCES : 3  ,         
    MAX_RESOURCES : 12 ,         
    MIN_DEPENDENCIES : 2  ,      
    MAX_DEPENDENCIES : 8  ,      
    MIN_REWARD : 5  ,            
    MAX_REWARD : 110,            
    MIN_DIFFICULTY : 20 ,        
    MAX_DIFFICULTY : 80 ,        
    MIN_PROBLEMS_PROB : 0.2,     
    MAX_PROBLEMS_PROB : 0.5,     

    NUMBER_OF_TASKS : 50,
    NUMBER_OF_WORKERS : 10
}
param_sets.append(explorator_pm)

#### - Equipo con alta motivación y baja pereza
Este conjunto simula un equipo extremadamente motivado y con poca pereza, lo que debería conducir a una alta productividad.

In [6]:
# Parámetros de los agentes trabajadores
hight_motivation = {
    MIN_PS_AGENTS : 30 ,         
    MAX_PS_AGENTS : 90 ,         
    MIN_FRIENDSHIP : 0.6,        
    MAX_FRIENDSHIP : 1.0,        
    MIN_MIN_ENERGY : 40 ,        
    MAX_MIN_ENERGY : 90 ,        
    MIN_MOTIVATION : 50 ,        # Alta motivación
    MIN_LAZZY : 0.0,             # Baja pereza
    MAX_LAZZY : 0.1,             

    # Parámetros del Project Manager      
    RISKY : 0.2,                 
    MIN_WORK_PROB : 0.0,         
    MAX_WORK_PROB : 0.2,         
    MIN_COOPERATION : 0.6,       
    MAX_COOPERATION : 1.0,       
    MIN_EXPLORATION : 0.0,       
    MAX_EXPLORATION : 0.2,       

    # Parámetros de las tareas
    MIN_DURATION : 5  ,          
    MAX_DURATION : 80 ,          
    INTERVAL : 1.1,              
    MIN_RESOURCES : 2  ,         
    MAX_RESOURCES : 10 ,         
    MIN_DEPENDENCIES : 1  ,      
    MAX_DEPENDENCIES : 7  ,      
    MIN_REWARD : 20 ,            
    MAX_REWARD : 100,            
    MIN_DIFFICULTY : 15 ,        
    MAX_DIFFICULTY : 70 ,        
    MIN_PROBLEMS_PROB : 0.1,     
    MAX_PROBLEMS_PROB : 0.4,     

    NUMBER_OF_TASKS : 60,
    NUMBER_OF_WORKERS : 10
}
param_sets.append(hight_motivation)

### Definimos la simulación

In [7]:
import random
from PMOntologic.Opportunity import *
from PMOntologic.PMO import Project
from PMOntologic.Resource import Resource
from Simulation.Simulator import Simulator
from Simulation.Environment import WorkCenter
from Simulation.PMAgent import PMAgent, PMperception
from Simulation.WorkerAgent import WorkerAgent
from Tasks.GeneticAlgorithm.Population import Population
from Tasks.GeneticAlgorithm.Tasks_combination import optimization_function
from Tasks.task import Task
from PMOntologic.Risk import *

for param in param_sets:
    agents = []

    for i in range(param[NUMBER_OF_WORKERS]):
        agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(param[MIN_PS_AGENTS],param[MAX_PS_AGENTS]), friendship=random.uniform(param[MIN_FRIENDSHIP],param[MAX_FRIENDSHIP]))
        agents.append(agent)

    resources_id = ['budget','Resource1', 'Resource2', 'Resource3', 'Resource4', 'Resource5', 'Resource6', 'Resource7', 'Resource8', 'Resource9', 'Resource10']
    resources_cant = {resource : 0 for resource in resources_id}

    # Definamos tareas de modo que no queden dependencias circulares
    tasks = []
    for i in range(param[NUMBER_OF_TASKS]):
        task = Task(id=i, priority=random.randint(0,5), duration=random.randint(param[MIN_DURATION],param[MAX_DURATION]), reward=random.randint(param[MIN_REWARD],param[MAX_REWARD]), difficulty=random.randint(param[MIN_DIFFICULTY],param[MAX_DIFFICULTY]), problems_probability=random.uniform(param[MIN_PROBLEMS_PROB],param[MAX_PROBLEMS_PROB]))
        cant = random.randint(0,10)
        for _ in range(cant):
            r = random.choice(resources_id)
            amount = random.randint(0,5)
            resource = Resource(r, amount)
            task.resources.append(resource)
            resources_cant[r] += amount 
        tasks.append(task)

    for i in range(param[NUMBER_OF_TASKS] - 1):
        if random.random() < 0.2 :
            dep = random.randint(int(param[MIN_DEPENDENCIES]),int(param[MAX_DEPENDENCIES]))
            for j in range(dep):
                index = random.randint(i+1, param[NUMBER_OF_TASKS] - 1)
                tasks[i].dependencies.append(tasks[index])

    risks = [ExecutionDelayRisk(), AcceleratedSpendingRisk(), CostIncreaseRisk(), UnfulfilledDependencesRisk(), LowProductivityRisk(), LowMotivationRisk(), LowPriorityRisk()]
    opportunities = [HighProductivityOpportunity(), HighMotivationOpportunity(), OptimizedResourcesOpportunity(), CostSavingOpportunity(), EarlyCompletionOpportunity(), HighCooperationOpportunity(), MilestoneCompletionOpportunity(), RiskManagementSuccessOpportunity(), InnovationOpportunity()]

    project = Project(objective='number_of_tasks',tasks=tasks, resources=[Resource(resource,cant*1.2) for resource,cant in resources_cant.items()], risks=risks, opportunities=opportunities)

    pm = PMAgent(min_motivation_team=20, initial_perception=PMperception(actual_time=0, team_motivation=80), project=project)
    pm.know_workers([(agent.id,agent.problem_solving) for agent in agents])

    simulator = Simulator(WorkCenter)
    simulations = simulator.StartManySimulationsThreading(8,1000,agents,pm,project)  
    for sim in simulations:
        sim.save_log_to_csv()
    # env = simulator.StartSimulation(1000, agents, pm, project )  
    # env.save_log_to_csv()  
